# Week 4 

# Peer-graded Assignment: Capstone Poject - The Battle of Neighborhoods (First Week) 


## Educational Objectives

This Notebook will be used for the final project. In this first week, I state the introduction, background, motivation, scope definition, data & methodology used. This capstone project is done by Conrado Montealegre.



<h1 id="Title: Comparison of two small cities (towns) in the Northern Sierra of Puebla - Zacatlán vs Chignahuapan">Title: Comparison of two small cities (towns) in the Northern Sierra of Puebla - Zacatlán vs Chignahuapan</h1>

<h2 id="1. Introduction (motivations and justification, business/social problem)">1. Introduction (motivations and justification, business/social problem)</h2>

In this work, I will compare two small cities (towns) located in the country province of Mexico on the so-called Northern Sierra of Puebla in the middle of Mexico. Zacatlan and Chinahuapan are the names of these towns. Both are located in a geographical zone which is a transition zone between the central Mexican plateau, and the coastal zone of the Gulf of Mexico. This transition is characterized by a complicated orography with mountains and ravines that hardens access to the more isolated country population. In fact, both cities are municipal heads. So, although these small cities are reasonably developed, they have some populations that live in rural isolated small communities.

But if all that has been stated is not interesting enough, even though both cities are close neighbors, they are not twin cities. Their historical governments have defined a particular development for each of them. Zacatlan has been historically a neural point for the trade routes between central Mexico (Mexico City and Puebla) and the coastal zone of the Gulf of Mexico (Veracruz). In addition, nowadays Zacatlan rulers have been trying to make this city an attractive tourism landmark. So, the downtown of Zacatlan has been changed from a typical rural Mexican town into a place with nice amenities with the purpose to attract foreign tourism.

On the other side, Chignahuapan government has impulsed industrial and economical development. Chigahuaán is also trying to attract tourism, but a different kind which could arrive and purchase the local products. One of these products is the production of "Christmas spheres" and other Christmas decorations. This economical activity is maybe one of the most important across Mexico in this area.

So, the purpose of this work is to explore these similitudes and differences. I will try to state how the local population's way of life can be affected by the different development plans in two cities close enough to make a comparison. I expect to find evidence about how the amount and type of amenities are a consequence of the different economical activities.

![Sierra](images/Sierra.jpg)

<h2 id="2. Data">2. Data</h2>

For this project, I will use 

* The data available on the internet, both on commercial websites and on government websites
* The official data from the Mexican National Institute of Statistics Geography and Information (INEGI, by its acronym in Spanish). 
* Foursquare app for collecting information about the different landmarks across Zacatlan and Chignahuapan.







![Sierra](images/INEGI.jpg)

<h2 id="3. Methodology">3. Methodology</h2>

The methodology that will be used is the same that we have been using across the course. First, I will download, process, and clean the data available. Later, I will perform an exploratory analysis of the data collected in order to gain insights into them. In this stage, I will visualize data mixed with geodata.

I will use unsupervised analysis (clustering) to better identified any pattern in the geo data available. I will try to identify different classes of landmarks with the purpose of characterizing the socioeconomic orientation of both towns.

Finally, I will analyze and discuss the findings train to state if the different economical activities of these cities really determine the class and amount of amenities and other landmarks.



![Sierra](images/Zacatlan.jpg)

![Sierra](images/Chignahuapan.jpg)

<h3 id="3.1. Getting/Creating Data">3.1. Getting/Creating Data</h3>

### Importing libraries

In [19]:
from config import CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION, LIMIT
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from geopy.geocoders import Nominatim


### Getting coordinates for Zacatlan and Chignahuapan

In [50]:
locator = Nominatim(user_agent="myGeocoder")

# Getting coordinates for Zacatlan
#str_Zacatlan = 'Zacatlan, Puebla, Mexico'
str_Zacatlan = 'Zacatlan, Puebla, Mexico'
zacatlan_loc = locator.geocode(str_Zacatlan)
print("{} Latitude = {}, Longitude = {}".format(str_Zacatlan,
zacatlan_loc.latitude, zacatlan_loc.longitude))
zacatlan_center = [zacatlan_loc.latitude, zacatlan_loc.longitude]

# Getting coordinates for Chignahuapan
str_Chignahuapan = 'Chignahuapan, Puebla, Mexico'
chignahuapan_loc = locator.geocode(str_Chignahuapan)
print("{} Latitude = {}, Longitude = {}".format(str_Chignahuapan,
chignahuapan_loc.latitude, chignahuapan_loc.longitude))
chignahuapan_center = [chignahuapan_loc.latitude, chignahuapan_loc.longitude]

Zacatlan, Puebla, Mexico Latitude = 19.96059515, Longitude = -98.00572136341704
Chignahuapan, Puebla, Mexico Latitude = 19.8213915, Longitude = -98.11775714603566


### Map of Zacatlan

In [23]:
import folium

In [51]:
map_zacatlan = folium.Map(location=zacatlan_center, zoom_start=15)
folium.Marker(zacatlan_center, popup='Zacatlán').add_to(map_zacatlan)
map_zacatlan

### Map of Chignahuapan

In [30]:
map_chignahuapan = folium.Map(location=chignahuapan_center, zoom_start=15)
folium.Marker(chignahuapan_center, popup='Chignauapan').add_to(map_chignahuapan)
map_chignahuapan